# Tutorial
---

Last run: 31/01/2024

Here we provide some examples of how to create images and populate them with model sources and skies.  This notebook is split into the following sections.

 1. **Basic image creation**
 2. **Masking an image and fitting the sky background**
 3. **Advanced technique: making an average sky model with dithered exposures**

## 1. Basic image creation
---

In [ ]:
import numpy as np
from astropy.table import Table
from astropy.convolution import Gaussian2DKernel
from astropy.stats import sigma_clipped_stats as scs
from astropy.modeling.models import Legendre2D
import cv2
import galsim
import matplotlib.pyplot as plt
from matplotlib import patches
from fakes import insert_fakes as insfks
from fakes import reduction_pipeline as rp

This section shows the fundamentals behind creating images using our software package.

### Setting up a blank image

We first have to set up a blank image object, which we'll later populate with fake sources.  We start by setting up the parameters of that image.  Some of these we won't need until later.

In [ ]:
magZp = 33.1  # Photometric zeropoint, in AB magnitudes

bnFac = 64  # Factor for binning during polynomial sky estimation
polyDeg = 1  # Order of polynomial for sky models

raCen = 150.040635  # Field central right ascension, deg
decCen = 2.208592  # Field central declination, deg
pxScale = 0.2  # arcsec/px
dimX = 2048  # Size of image x-axis
dimY = 2048  # Size of image y-axis

fwhm = 0.7  # Star PSF FWHM, in arcseconds

# Set up the parameters of a sky model as a dictionary
skyMean = 10**(-0.4*(20 - magZp - 2.5*np.log10(pxScale**2)))  # 20 mag/arcsec^2 surface brightness
skyDict = {'c0_0': skyMean,
           'c1_0': 0.015,
           'c0_1': 0.0056,
           'c1_1': 0.0}

noise = galsim.noise.CCDNoise(gain=3., read_noise=5.)  # Noise generator; units are counts

Now we create the image object that we'll use as a canvas.

In [ ]:
image = insfks.ImageBuilder(dimX, dimY,
                            raCen, decCen,
                            pxScale,
                            skyDict,
                            None)
# The last item is noise.  If we include it here, the image will have instrumental noise added to it immediately.
# But we will include it later for demonstration purposes.

### Creating a source catalogue

Time to populate the image with some fake sources.  To keep things simple, we'll only use stars for this example.

In [ ]:
# We wrote a function to grab star parameters from the SDSS
rad = np.round((1.5*dimX*pxScale)/3600, 1)  # Needs a search radius in degrees; we'll use a large one for insurance
sdss_cat = insfks.getSDSSCatalogue(raCen, decCen, rad)

In [ ]:
# Turn it into a more accessible table, with AB magnitudes
def sdssToAb(mag):
    A = -(np.log(10)/2.5)
    b = 1.8*10**-10  # i-band value
    fJy = np.sinh(A*mag - np.log(b))*2*b
    abMag = -2.5*np.log10(fJy)

    return abMag

# Accept only valid i-band magnitudes
want = (sdss_cat['psfMag_i'] > 0) \
        & np.isfinite(sdssToAb(sdss_cat['psfMag_i']))
# Make a new table
sdss_stars = Table([sdssToAb(sdss_cat['psfMag_i'][want]),
                    sdss_cat['ra'][want],
                    sdss_cat['dec'][want]],
                    names=['mag', 'ra', 'dec'])

### Populate the image with fakes

Now that we have a blank image and a star catalogue, we can inject some star models into the image.

In [ ]:
for k in range(len(sdss_stars)):
    mod_star = insfks.DrawModels(ra=sdss_stars['ra'][k],
                                 dec=sdss_stars['dec'][k],
                                 image=image.image)  # This is the image object we draw in
    mod_star.drawPsf(beta=3,
                    fwhm=fwhm,
                    mag=sdss_stars['mag'][k],
                    method='auto',
                    magZp=magZp)

In [ ]:
# Let's take a quick look
fig, ax = plt.subplots(1, figsize=(5, 5))
ax.imshow(image.image.array, vmin=-10, vmax=10)

### Add a model sky and some noise

Now we can add a plane sky to our image, plus some instrumental and shot noise.  We already set these up, so we just have to generate and add them.

In [ ]:
image.makeModelSky(polyDeg=polyDeg)  # Run this method to generate the sky model
image.image.addNoise(noise)  # This adds instrumental noise to the image array, using the noise model

In [ ]:
fullIm = image.image.array + image.sky  # Add on the sky

In [ ]:
# Now let's look at the full image
fig, ax = plt.subplots(1, figsize=(5, 5))
ax.imshow(fullIm, vmin=skyMean-100, vmax=skyMean+100)

## 2. Masking an image and fitting the sky background
---

Now that we have an image to work with, we can try modelling the sky background and removing it.  Since we know the sources we injected, we don't have to run any detection algorithm to generate the mask.  Instead, we'll make a noise-free image with only the stars in it and use that.

In [ ]:
# New image, no sky and noise.  Then populate it with stars.  See above.
image = insfks.ImageBuilder(dimX, dimY,
                            raCen, decCen,
                            pxScale,
                            {},  # Just passing a blank sky dictionary.
                            None)  # No noise.

for k in range(len(sdss_stars)):
    mod_star = insfks.DrawModels(ra=sdss_stars['ra'][k],
                                 dec=sdss_stars['dec'][k],
                                 image=image.image)
    mod_star.drawPsf(beta=3,
                    fwhm=fwhm,
                    mag=sdss_stars['mag'][k],
                    method='auto',
                    magZp=magZp)

We'll mask all star models down to 32 mag/arcsec^2 in surface brightness.

In [ ]:
# The masking function wants an HDUList, so we'll make that first.
imHdu = rp.makeHduList(image.image.array)

mask = rp.maskToLimit(imHdu,
                      sbLim=32,
                      magZp=magZp,
                      pxScale=pxScale)

In [ ]:
# Applying the mask and visualizing the results
maskedIm = fullIm * 1.0
maskedIm[mask[0].data > 0] = np.nan
maskedIm = rp.makeHduList(maskedIm)  # We'll need it in this format

fig, ax = plt.subplots(1, figsize=(5, 5))
ax.imshow(maskedIm[0].data, vmin=skyMean-100, vmax=skyMean+100)

Every white pixel (NaN) is masked.  Now we can fit a plane to this masked image.  We also have a function for this.

In [ ]:
skyFit, m, fit = rp.legendreSkySub(polyDeg,
                                   maskedIm,
                                   bnFac=bnFac,
                                   full=True)  # This will allow us to look at the full fit parameters

# We'll produce an image of the true sky model for comparison
trueM = Legendre2D(polyDeg, polyDeg,
                   c0_0=skyDict['c0_0'],
                   c0_1=skyDict['c0_1'],
                   c1_0=skyDict['c1_0'],
                   c1_1=skyDict['c1_1'])
X, Y = np.meshgrid(np.arange(1, dimX+1),
                   np.arange(1, dimY+1))
trueSky = trueM(X, Y)
# Ignore all the warnings.

Let's look at the best-fit sky model.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(skyFit, vmin=skyMean-50, vmax=skyMean+50)
ax[0].set_title(r"Best-fit")
ax[1].imshow(trueSky, vmin=skyMean-50, vmax=skyMean+50)
ax[1].set_title(r"True sky")
ax[2].imshow(trueSky - skyFit, vmin=-1, vmax=1)
ax[2].set_title(r"True sky $-$ Best-fit")

There are some slight differences.  We can check the best-fit parameters against the input values.

In [ ]:
# Making a dictionary
fitSkyDict = {"c0_0": m.c0_0.value,
              "c1_0": m.c1_0.value,
              "c0_1": m.c0_1.value,
              "c1_1": m.c1_1.value}

In [ ]:
# Looking at both side-by-side, the input and the best-fit
for key in skyDict.keys():
    print(key + "   %.4f %.4f" % (skyDict[key], fitSkyDict[key]))

## 3. Advanced technique: making an average sky model with dithered exposures
---

Here we demonstrate something more complex: the creation of a series of dithered "exposures", each with the same model sky injected into them, which we then median-combine and smooth to generate an averaged sky model.  The quickest way to do this is to sample sub-images from a larger, master image, rather than populating separate images with new models each iteration.

In [ ]:
# Creating the larger, master image
mSize = dimX*2
masterIm = insfks.ImageBuilder(mSize, mSize,  # Twice as big as our initial test image
                               raCen, decCen,  # Same center
                               pxScale,  # Same scale
                               {},  # No sky on the master; we'll add these to each exposure
                               None)  # No noise either

# Now adding stars to it
for k in range(len(sdss_stars)):
    mod_star = insfks.DrawModels(ra=sdss_stars['ra'][k],
                                 dec=sdss_stars['dec'][k],
                                 image=masterIm.image)
    mod_star.drawPsf(beta=3,
                    fwhm=fwhm,
                    mag=sdss_stars['mag'][k],
                    method='auto',
                    magZp=magZp)

In [ ]:
fig, ax = plt.subplots(1, figsize=(5, 5))
ax.imshow(masterIm.image.array, vmin=-1, vmax=10)

Now we can generate a list of image coordinates.  Our code uses a 3 x 3 grid of pointings around the central coordinate, but randomness can be added to these coordinates using the `tol` parameter, which is the fraction of the "exposure" size by which to perturb the default 9-point grid coordinates.

We'll make 10 total exposures, then combine them to make our sky estimate.

In [ ]:
def makeSubIm(masterImage, xCen, yCen, size):
    '''
    Function to grab a smaller image from a larger one
    '''
    sub_im = masterImage.image.array[yCen - size//2: yCen + size//2,
                                     xCen - size//2: xCen + size//2]
    sub_im = rp.makeHduList(sub_im)

    return sub_im

In [ ]:
# Generating the "exposure" coordinates
ditherStep = dimX//4  # We'll use 1/4 the exposure image size for the 9-point pattern positions
tol = 0.05  # And a 5% random offset in x and y from those coordinates

# Center of the master image; we'll need this later
xCen = masterIm.image.array.shape[1]//2
yCen = masterIm.image.array.shape[0]//2

# This is required because the self.ditheredCoordinates() method
# blanks the image, so we want a blank image with the same WCS for that method
masterBase = insfks.ImageBuilder(mSize, mSize,
                                 raCen, decCen,
                                 pxScale,
                                 {})

# 10 total exposures
N = 10
ims = []  # We'll append our new exposures to this
xcens = []
ycens = []  # Saving these for display purposes
for i in range(N):
    masterBase.ditheredCoordinates(ditherStep, tol)

    # Now we'll adjust these in case the borders of some land outside of the master image boundaries
    offsetX, offsetY = masterBase.offsets  # Grabbing the dithered positions
    x_cen = xCen + offsetX
    y_cen = yCen + offsetY
    # We're making these sub-images the same size as the test images above
    if (x_cen > mSize - dimX//2):
        x_cen = mSize - dimX//2
    if (x_cen < dimX//2):
        x_cen = dimX//2
    if (y_cen > mSize - dimY//2):
        y_cen = mSize - dimY//2
    if (y_cen < dimY//2):
        y_cen = dimY//2
    xcens.append(x_cen)
    ycens.append(y_cen)
        
    # Generating a sky with same params but different noise each time
    # Sky
    skyBase = insfks.ImageBuilder(dimX, dimY,
                                  raCen, decCen,
                                  pxScale,
                                  skyDict)  # We'll use the same sky model from before
    skyBase.makeModelSky(polyDeg)
    
    # Noise
    noiseIm = galsim.Image(dimX, dimY)
    noiseIm.addNoise(noise)

    subImage = makeSubIm(masterIm, x_cen, y_cen, dimX)  # Take the sub-image from the true master
    fullImage = subImage[0].data + skyBase.sky + noiseIm.array

    ims.append(fullImage)

In [ ]:
# Let's visualize where these image cutouts all landed
fig, ax = plt.subplots(1, figsize=(5, 5))
ax.imshow(masterIm.image.array, vmin=-1, vmax=10)
for i in range(len(xcens)):
    pch = patches.Rectangle((xcens[i]-dimX//2,
                             ycens[i]-dimY//2),  # This needs the bottom-left corner coordinate
                           width=dimY,
                           height=dimX,
                           fill=False,
                           edgecolor="w",)
    ax.add_patch(pch)

In [ ]:
# And verify that some of our exposures came out correctly
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(ims[0], vmin=skyMean-100, vmax=skyMean+100)
ax[1].imshow(ims[5], vmin=skyMean-100, vmax=skyMean+100)
ax[2].imshow(ims[9], vmin=skyMean-100, vmax=skyMean+100)

All of our test exposures have the same sky model, but are placed in different locations around the master image.  Now we can make a sky estimate by median-combining these dithered exposures, and compare it to the true input sky model.

In [ ]:
# We need a smoothing kernel, to reduce the noise in the combined sky model.
kernelWidth = (1/15)*dimX  # Same scale as CFHT approach
kernel = Gaussian2DKernel(x_stddev=kernelWidth,
                          y_stddev=kernelWidth).array

In [ ]:
# Now average the images and smooth the result
__, avSky, __ = scs(ims, sigma=3, maxiters=5, axis=0)
avSky = cv2.filter2D(avSky, -1, kernel)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(avSky, vmin=skyMean-50, vmax=skyMean+50)
ax[0].set_title(r"Averaged, N=%i" % (N))
ax[1].imshow(trueSky, vmin=skyMean-50, vmax=skyMean+50)
ax[1].set_title(r"True sky")
ax[2].imshow(trueSky - avSky, vmin=-5, vmax=5)
ax[2].set_title(r"True sky $-$ Averaged")

As demonstrated in Watkins et al. (2024), the averaged sky has some noise leftover on the convolution kernel scale (as well as some problems at the image edges), but is fairly close to the input sky.